In [1]:
import pandas as pd
import cv2
import os
from keras.layers import Dense, Flatten
from keras.models import Model
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical
from keras.applications.resnet50 import ResNet50
import matplotlib.pyplot as plt
# print('CPU:', tf.config.list_physical_devices(device_type='CPU'))
# print('GPU:', tf.config.list_physical_devices(device_type='GPU'))

In [2]:
file_path = './sfddd/driver_imgs_list.csv'
df = pd.read_csv(file_path)
df.head()

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [3]:
IMAGE_SIZE = (224,224)
# num: how many images in sequence do you want to load
def load_train_data(num):
    images = []
    labels = []
    dataset = './sfddd/imgs/train'
    for folder in os.listdir(dataset):
        label = folder
        subdir = f'{dataset}/{folder}'
        for file in tqdm(os.listdir(subdir)[:num]):
            img_path = os.path.join(subdir, file)
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, IMAGE_SIZE) # to match ResNet
            images.append(image)
            labels.append(int(label[-1]))
    images = np.array(images)
    labels = np.array(labels)
    return [images, labels]

def load_test_data(num):
    images = df['img'][:num]
    labels = df['classname'][:num]
    print(images.shape)
    print(labels[:5])
    dataset = './sfddd/imgs/test'
    for i, img in enumerate(images):
        image_path = f'{dataset}/{img}'
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, IMAGE_SIZE) # to match ResNet
        images[i] = (image)
        labels[i] = int(labels[i][-1])
    images = np.array(images)
    labels = np.array(labels)
    return [images, labels]

In [4]:
X, y = load_train_data(100)
y = to_categorical(y)
print(len(X), len(y))
print(X.shape, y.shape)

100%|██████████| 100/100 [00:00<00:00, 375.77it/s]


1000 1000
(1000, 224, 224, 3) (1000, 10)


In [8]:
# ResNet
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
for layer in model.layers:
    layer.trainable = False
x = Flatten()(model.output)
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(10, activation='softmax')(x)
model = Model(inputs=model.input, outputs=outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=10, batch_size=100)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.1048 - loss: 181.2718
Epoch 2/10
 3/10 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.1356 - loss: 58.2269

KeyboardInterrupt: 

In [26]:
X_test, y_test = load_test_data(500)
y_test = to_categorical(y_test)

(500,)
0    c0
1    c0
2    c0
3    c0
4    c0
Name: classname, dtype: object


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
